In [1]:
import os

from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, df_tangara_sensors, df_to_csv

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from aqi_epa_pm25.ipynb
PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Tangara Sensors

In [2]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME=os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME=os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')

Since: 2023-10-05T12:51:16-05:00 -> 1696528276000, Until: 2023-10-05T20:51:16-05:00 -> 1696557076000


In [3]:
# Data Frame Tangaras from InfluxDB
df_tangaras = df_tangara_sensors(start_timestamp, end_timestamp)
df_tangaras.drop_duplicates(subset=['MAC'], inplace=True)

print(f"Period of Time: Since: {START_ISO8601_DATETIME}, Until: {END_ISO8601_DATETIME}")
print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Period of Time: Since: 2023-10-05T12:51:16-05:00, Until: 2023-10-05T20:51:16-05:00
Total Tangara Sensors: 13


,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
ID,,,,,
TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.3844757080078125,-76.51634216308594
TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.4613800048828125,-76.51222229003906
TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.4586334228515625,-76.46415710449219
TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.4517669677734375,-76.55067443847656
TANGARA_2B42,d29e6pg,D29ESP32DED2B42,3.4270477294921875 -76.54792785644531,3.4270477294921875,-76.54792785644531


In [4]:
# Save Tangaras into CSV file
df_to_csv(df_tangaras, "tangaras.csv")